&emsp;&emsp;对于分类问题而言,给定一个训练样本集,求比较粗糙的分类规则(弱分类器)要比求精确的分类规则(强分类器)容易得多.
提升方法就是从弱学习算法出发,反复学习,得到一系列弱分类器(又称为基本分类器),然后组合这些弱分类器,构成一个强分类器.
大多数的提升方法都是改变训练数据的概率分布(训练数据的权值分布),针对不同的训练数据分布调用弱学习算法学习一系列弱分类器.         
&emsp;&emsp;这样,对提升方法来说,有两个问题需要回答:一是在每一轮如何改变训练数据的权值或概率分布;二是如何将弱分类器组合成一个强分类器.
关于第一个问题,AdaBoost的做法是,提高那些被前一轮分类器错误分类样本的权值,而降低那些被正确分类样本的权值.这样一来,
那些没有得到正确分类的数据,由于其权值的加大而受到后一轮的弱分类器的更大关注.于是,分类问题被一系列的弱分类器"分而治之".至于第二个问题,
即弱分类器的组合,AdaBoost采取加权多数表决的方法,具体地,加大分类误差小的弱分类器的权值,使其在表决中起较大的作用;
减少分类误差大的弱分类器的权值,使其在表决中起较小的作用.          
&emsp;&emsp;AdaBoost的巧妙之处就在于它将这些想法自然且有效地实现在一种算法里.                     

### AdaBoost模型的推导

&emsp;&emsp;AdaBoost算法有多种推导方式,比较容易理解的是基于"加法模型"(additive model),即基本分类器的线性组合      
$$  f(\mathbf{x}) = \sum_{m=1}^T \alpha_t G_t(\mathbf{x}) \tag{1.0}$$            
来最小化指数损失函数(exponential loss function)($ \mathcal{D} $表示训练数据集$ D $的分布)     
$$ L_{exp}(y,f(\mathbf{x})| D ) = E_{x \sim \mathcal{D}}[e^{-yf(\mathbf{x})}] \tag{1.1} $$            
其中,$\mathbf{x}_i \in \mathcal{X} \subseteq \mathbb{R}^n, y_i \in \mathcal{Y}=\{-1, 1\}$,$\alpha_t$为基本分类器$G_t(\mathbf{x})$的系数.
可以看出前向分步算法逐一学习基函数,这一过程与AdaBoost算法逐一学习基本分类器的过程一致,所以说AdaBoost算法是前向分步加法算法的特例.       
&emsp;&emsp;若$f(\mathbf{x})$能令指数损失函数最小化,则考虑式(1.1)对$f(\mathbf{x})$的偏导              
$$ \frac{\partial L_{exp}(y,f(\mathbf{x})|\mathcal{D})}{\partial f(\mathbf{x})} = -e^{-f(\mathbf{x})} P(y=1|x) + e^{f(\mathbf{x})} P(y=-1|\mathbf{x}) \tag{1.3} $$         
令式(1.3)为零可解得            
$$ f(\mathbf{x}) =\frac{1}{2} \ln \frac{P(y=1 |\mathbf{x})}{P(y=-1 | \mathbf{x})} \tag{1.4} $$
因此有          

$$
\begin{aligned}
\mathrm{sign}(f(\mathbf{x})) &=\mathrm{sign}\left(\frac{1}{2} \ln \frac{P(y=1 | \mathbf{x})}{P(y=-1 | \mathbf{x})}\right) \\
&=\left\{\begin{array}{ll}
1, & P(y=1 | \mathbf{x})>P(y=-1 | \mathbf{x}) \\
-1, & P(y=1 | \mathbf{x})<P(y=-1 | \mathbf{x}) \\
\end{array}\right. \\
& =\underset{y_i \in\{-1,1\}}{\arg \max } P(y_i | \mathbf{x})
\end{aligned}       
$$

这意味着$\mathrm{sign}(f(\mathbf{x}))  $达到了贝叶斯最优错误率.换言之,若指数损失函数最小化,则分类误差率也就最小化;
这说明指数损失函数是分类任务原本$ 0/1$损失函数的一致的(consistent)替代损失函数(一致性亦称"相合性").
由于这个替代损失函数有更好的数学性质,例如它是连续可微函数,因此我们用它代替$ 0/1 $损失函数作为优化目标.             
&emsp;&emsp;在AdaBoost算法中,第一个基本分类器$ G_1 $是通过直接将基学习算法用于初始数据分步而得;
此后迭代地生成$ G_t $和$\alpha_t$,当基本分类器$G_t$基于分布$ D_t $产生之后,
该基本分类器的权重$ \alpha_t $应使得$ \alpha_t G_t $最小化指数损失函数(权重$\alpha_t$与经过$t-1$轮迭代前向分步算法已经得到的$ f_{t-1}(\mathbf{x}) $无关)      

$$
\begin{aligned}
L_{exp}(\alpha_t G_t|\mathcal{D}) &= E_{x \sim \mathcal{D}} [e^{-y \alpha_t G_t(\mathbf{x})}] \\
							&= E_{x \sim \mathcal{D}}[e^{-\alpha_t \mathbb{I}(y=G_t(\mathbf{x})} + e^{\alpha_t \mathbb{I}(y \neq G_t(\mathbf{x})}] \\
							&=e^{-\alpha_t} P_{\mathbf{x} \sim \mathcal{D}_t}(y = G_t(\mathbf{x})) + e^{\alpha_t} P_{\mathbf{x} \sim \mathcal{D}_t}(y \neq G_t(\mathbf{x})) \\
							&=e^{-\alpha_t} (1-\epsilon_t) + e^{\alpha_t} \epsilon_t 
\end{aligned}               
$$

其中$ \epsilon_t =  P_{\mathbf{x} \sim \mathcal{D}_t}(y \neq G_t(\mathbf{x}))$.考虑指数损失函数的导数       
$$ \frac{L_{exp}(\alpha_t G_t|\mathcal{D})}{\partial \alpha_t} = -e^{-\alpha_t} (1-\epsilon_t) + e^{\alpha_t} \epsilon_t \tag{1.5} $$            
令式(1.5)为零可解得      
$$ \alpha_t= \frac{1}{2} \ln \left( \frac{1 - \epsilon_t}{\epsilon_t} \right) $$        
&emsp;&emsp;AdaBoost算法在获得$f_{t-1}(\mathbf{x})$之后样本分步将进行调整,使下一轮基本分类器$G_t$能纠正$f_{t-1}(\mathbf{x})$的一些错误.
理想的$G_t$能纠正$f_{t-1}(\mathbf{x})$的全部错误,即最小化                

$$
\begin{aligned}
L_{exp}(f_{t-1}(\mathbf{x}) + \alpha_t G_t|\mathcal{D}) &=  E_{\mathbf{x} \sim \mathcal{D}}[e^{-y(f_{t-1}(\mathbf{x}) + \alpha_t G_t(\mathbf{x}))}] \\
											&= E_{\mathbf{x} \sim \mathcal{D}}[e^{-yf_{t-1}(\mathbf{x})}  e^{-y \alpha_t G_t(\mathbf{x})} ]  
\end{aligned}
\tag{1.6}
$$     

注意到$y^2 = G_t^2(\mathbf{x})=1$式(1.6)可使用$e^{-y\alpha_t G_t(\mathbf{x})}$的泰勒近似(易知展开为二阶和展开为二阶以上没有区别)为               

$$
\begin{aligned}
L_{exp}(f_{t-1}(\mathbf{x}) + \alpha_t G_t|\mathcal{D}) &\cong  E_{\mathbf{x} \sim \mathcal{D}} \left[e^{-y f_{t-1}(\mathbf{x})} \left( 1 - y \alpha_t G_t(\mathbf{x}) + \frac{y^2 \alpha_t^2 G_t^2(\mathbf{x})}{2} \right) \right] \\
												&= E_{\mathbf{x} \sim \mathcal{D}} \left[e^{-y f_{t-1}(\mathbf{x})} \left( 1 - y \alpha_t G_t(\mathbf{x}) +  \frac{\alpha_t^2}{2} \right) \right] 
\end{aligned}  
$$   

于是,理想的基本分类器        

$$
\begin{aligned}
G_t(\mathbf{x}) &= \mathrm{arg} \min_{G} L_{exp}(f_{t-1}(\mathbf{x}) + \alpha_t G|\mathcal{D}) \\
				&=  \mathrm{arg} \min_{G}  E_{\mathbf{x} \sim \mathcal{D}} \left[e^{-y f_{t-1}(\mathbf{x})} \left( 1 - y \alpha_t G(\mathbf{x}) +  \frac{\alpha_t^2}{2} \right) \right]  \\
				&=  \mathrm{arg} \max_{G}  E_{\mathbf{x} \sim \mathcal{D}} \left[e^{-y f_{t-1}(\mathbf{x})}  y \alpha_t G(\mathbf{x})  \right]  \qquad 由上知:\alpha_t 与\mathcal{D}无关(与\mathcal{D}_t相关)  \\
				&=  \mathrm{arg} \max_{G}  E_{\mathbf{x} \sim \mathcal{D}} \left[ \frac{e^{-y f_{t-1}(\mathbf{x})}}{E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t-1}(\mathbf{x})} ]}  y \alpha_t G(\mathbf{x})  \right]  \\ 
				&=  \mathrm{arg} \max_{G}  E_{\mathbf{x} \sim \mathcal{D}} \left[ \frac{e^{-y f_{t-1}(\mathbf{x})}}{E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t-1}(\mathbf{x})} ]}  y G(\mathbf{x})  \right]  \qquad 易知:\alpha_t >0(因为\alpha_i为基本分类器权重)
\end{aligned} 
$$                

注意到$ E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t-1}(\mathbf{x})} ]  $是一个常数,令$\mathcal{D}_t$表示一个分布       
$$ \mathcal{D}_t(\mathbf{x}) = \frac{e^{-yf_{t-1}(\mathbf{x})}}{ E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t-1}(\mathbf{x})} ]} $$            
根据数学期望的定义,这等价于令                   

$$
\begin{aligned}
G_t(\mathbf{x}) &= \mathrm{arg} \max_{G}  E_{\mathbf{x} \sim \mathcal{D}}  \left[ \frac{e^{-yf_{t-1}(\mathbf{x})}}{ E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t-1}(\mathbf{x})} ]}    y  G_t(\mathbf{x})  \right]  \\
				&= \mathrm{arg} \max_{G}  E_{\mathbf{x} \sim \mathcal{D}_t} [y  G(\mathbf{x})] 
\end{aligned}  
$$   

由$ y,G(\mathbf{x})  \in \{-1, +1 \}$,有                
$$  yG(\mathbf{x}) = 1 - 2 \mathbb{I}(y \neq G(\mathbf{x}))  $$          
则理想的基本分类器    
$$ G_t(\mathbf{x}) = \mathrm{arg} \min_{G}  E_{\mathbf{x} \sim \mathcal{D}_t} [ \mathbb{I} (y \neq G(\mathbf{x})) ]   $$            
由此可见,理想的$G_t$将在分布$\mathcal{D}_t$下最小化分类误差.因此,基本分类器将基于分布$\mathcal{D}_t$来训练,
且针对$\mathcal{D}_t$的分类误差应小于0.5(随机猜测).这在一定程度上类似"残差逼近"的思想.
考虑$\mathcal{D}_t$和$\mathcal{D}_{t+1}$的关系,有            

$$
\begin{aligned}
\mathcal{D}_{t+1}(\mathbf{x}) &= \frac{e^{-yf_{t}(\mathbf{x})}}{ E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t}(\mathbf{x})} ]} \\ 
							  &= \frac{e^{-yf_{t-1}(\mathbf{x})}e^{-y \alpha_t G_t(\mathbf{x})}}{ E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t}(\mathbf{x})} ]}  \\
							  &=  \frac{e^{-yf_{t-1}(\mathbf{x})}}{ E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t-1}(\mathbf{x})} ]}   e^{-y \alpha_t G_t(\mathbf{x})}  \frac{ E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t-1}(\mathbf{x})} ]  }{ E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t}(\mathbf{x})} ]} \\
							  & =  \mathcal{D}_t(\mathbf{x})   e^{-y \alpha_t G_t(\mathbf{x})}   \frac{ E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t-1}(\mathbf{x})} ]  }{ E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t}(\mathbf{x})} ]}	\\
							  & = 	\mathcal{D}_t(\mathbf{x})   e^{-y \alpha_t G_t(\mathbf{x})} E_{\mathbf{x} \sim \mathcal{D}} \frac{ E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t-1}(\mathbf{x})} ]  }{ E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t-1}(\mathbf{x})}  e^{-y \alpha_t G_t(\mathbf{x})}]} \\
							  & = \frac{\mathcal{D}_t(\mathbf{x})   e^{-y \alpha_t G_t(\mathbf{x})} }{ E_{\mathbf{x} \sim \mathcal{D}}[ \frac{ e^{-y f_{t-1}(\mathbf{x})}  e^{-y \alpha_t G_t(\mathbf{x})}  }{ E_{\mathbf{x} \sim \mathcal{D}} e^{-f_{t-1}(\mathbf{x})} } ]  }	 \\
							  & =  \frac{\mathcal{D}_t(\mathbf{x})   e^{-y \alpha_t G_t(\mathbf{x})} }{ E_{\mathbf{x} \sim \mathcal{D}_t} [ \mathcal{D}_t(\mathbf{x})   e^{-y \alpha_t G_t(\mathbf{x})} ]  }  				  
\end{aligned}
$$            

令$ Z_t = \frac{ E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t}(\mathbf{x})} ]} {E_{\mathbf{x} \sim \mathcal{D}} [ e^{-y f_{t-1}(\mathbf{x})} ] } = E_{\mathbf{x} \sim \mathcal{D}_t} [ \mathcal{D}_t(\mathbf{x})   e^{-y \alpha_t G_t(\mathbf{x})} ]    $ ,
这里,$Z_t$是规范化因子.故上式可写为            

$$
\begin{aligned} 
\mathcal{D}_{t+1}(\mathbf{x})  &= \frac{\mathcal{D}_t(\mathbf{x})   e^{-y \alpha_t G_t(\mathbf{x})} }{Z_t} \\
							   &= \frac{\mathcal{D}_t(x)}{Z_t}  \times 
							   		\begin{cases}
										\exp(-\alpha_t), & \text{if} \quad G_t(\mathbf{x}) = y \\
        								\exp(\alpha_t), & \text{if} \quad  G_t(\mathbf{x}) \neq y 
     \end{cases}
\end{aligned} 
\tag{1.8}
$$           



<font color='red' size=4>算法(AdaBoost):</font> 

输入:训练数据集$D=\{(\mathbf{x}_1, y_1),  (\mathbf{x}_2, y_2), \cdots,(\mathrm{x}_N, y_N) \}  $,其中$\mathbf{x}_i \in \mathcal{X} \subseteq \mathbb{R}^n$,$y_i \in \mathcal{Y} =  \{-1, 1\}$;弱学习算法$\Phi$;训练轮数$T$  

输出:最终分类器$ H(x) $

1. $ \mathcal{D}_1(\mathbf{x}) = (w_{11},\cdots, w_{1i}, \cdots, w_{1N}), \quad w_{1i} = \frac{1}{N},\quad i=1,2,\cdots,N $
2.  **for** $ t=1,2,\cdots,T$ **do**
3. &emsp; &emsp;$G_t = \Phi(D, \mathcal{D}_t) $
4. &emsp;&emsp;$\epsilon_t = P_{\mathbf{x} \sim \mathcal{D}_t }(G_t(\mathbf{x}) \neq y)$
5. &emsp;&emsp;**if** $ \epsilon_t > 0.5 $   **then break**
6. &emsp;&emsp; $$  \alpha_t= \frac{1}{2} \ln \left( \frac{1 - \epsilon_t}{\epsilon_t} \right) \tag{1.7} $$
7. &emsp;&emsp;  

$$
\begin{aligned} 
   \mathcal{D}_{t+1}(\mathbf{x})  &= \frac{\mathcal{D}_t(\mathbf{x})   e^{-y \alpha_t G_t(\mathbf{x})} }{Z_t} \\
   							   &= \frac{\mathcal{D}_t(\mathbf{x})}{Z_t}  \times 
   							   		\begin{cases}
   										\exp(-\alpha_t), & \text{if} \quad G_t(\mathbf{x}) = y \\
           								\exp(\alpha_t), & \text{if} \quad  G_t(\mathbf{x}) \neq y
        \end{cases}
\end{aligned}
$$   

8. **end for**
9. 输出:$$ f(\mathbf{x}) = \left( \sum_{t=1}^T \alpha_t G_t(\mathbf{x}) \right); H(\mathbf{x}) = \mathrm{sign} (f(\mathbf{x}) )$$


<font color='red' size=4>算法(AdaBoost)说明:</font>    
步骤一:假设训练数据集具有均匀的权值分布,即每个训练样本在基本分类器的学习中作用形同,
这一假设保证第一步能够在原始数据上学习基本分类器$ G_1(\mathbf{x}) $      
步骤六:计算基本分类器$G_t(\mathbf{x})$的系数$\alpha_t$.$\alpha_t$表示$G_t(\mathbf{x})$在最终分类器中的重要性.
由式(1.8)可知,当$\epsilon_t \leq \frac{1}{2} $时,$\alpha_t \neq  0 $,并且$ \alpha_t $随着$ \epsilon_t $的减小而增大,
所以分类误差越小的基本分类器在最终分类器中的作用越大.         
步骤七:由式(1.7)可知,对于某条具体数据集$ \mathbf{x}_i ; y_i $    

$$
\begin{aligned} 
   \mathcal{D}_{t+1, i}(\mathbf{x})  &= \frac{\mathcal{D}_{t, i}(\mathbf{x})   e^{-y \alpha_t G_t(\mathbf{x})} }{Z_t} \\
   							   &= \frac{\mathcal{D}_{t,i}(\mathbf{x})}{Z_t}  \times 
   							   		\begin{cases}
   										\exp(-\alpha_t), & \text{if} \quad G_t(\mathbf{x}_i) = y_i \\
           								\exp(\alpha_t), & \text{if} \quad  G_t(\mathbf{x}_i) \neq y_i
        \end{cases}
\end{aligned}  
$$

由此可得,被基本分类器误分类样本的权值得以扩大,而被正确分类的权值却得以缩小.两者相比,
易知误分类样本的权值被放大$ e^{2 \alpha_t} = \frac{1 - \epsilon_t}{\epsilon} $倍.因此,误分类样本在下一轮学习中起更大的作用.
不改变所给的训练数据,而不断改变训练数据权值的分布,使得训练数据在基本分类器的学习中起不同的作用,这就是AdaBoost算法的一个特点.    
步骤九:线性组合$ f(\mathbf{x}) $实现了$T$个基本分类器的加权表决.系数$ \alpha_t $表示了基本分类器$G_t(\mathbf{x})$的重要性,这里,
所有$\alpha_t$之和并不为1.$ f(\mathbf{x}) $的符号决定实例$\mathbf{x}$的类,$ f(\mathbf{x}) $的绝对值表示分类的确信度(绝对值越大,确信度越高).